# AMath 584 Homework 2, Problem 8

Autumn Quarter, 2016.

Modified Gram-Schmidt illustrated in Python.

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


Note a few gotcha's:
- This first code works only if `A` is real.  See below for the complex case.
- If the user specifies `A` as an array of all integers then integer arithmetic is used and the results might be wrong.  In this code we explicitly convert to floats.
- `V` should be initialized to a copy of `A` somehow, either as done in this code or using `V = copy.copy(A)` as done in the complex case further below.  Simply setting `V = A` gives another pointer to the same array and so changing `V` would also change `A`.  For this particular code this wouldn't hurt, but in general can give unexpected results.
- The numpy function `inner` is used to take the inner product of two vectors that are 1d arrays, which is what the slicing operation `Q[:,i]` returns.

In [2]:
def mgs(A):
    """
    Modified Gram-Schmidt for real matrices.
    """
    # initialize V to be a copy of A, and insure data type is float
    # (in case user initializes with integers):
    V = array(A, dtype=float)  
    
    # Initialize Q, R of correct shapes, values filled in later:
    m,n = shape(A)
    Q = zeros((m,n))
    R = zeros((n,n))
    
    for i in range(n):
        R[i,i] = norm(V[:,i], 2)
        Q[:,i] = V[:,i]/R[i,i]
        for j in range(i+1,n):
            R[i,j]= inner(Q[:,i],V[:,j])
            V[:,j] = V[:,j] - R[i,j]*Q[:,i]
            
    return Q,R

Define a test function so we can easily test many cases:

In [3]:
def test_mgs(A):
    """
    Test the mgs function and compute the error in the computed Q*R
    relative to the original A.
    """
    Q,R = mgs(A)
    print "Q = \n",Q
    print "\nR = \n",R
    
    QR = dot(Q,R)
    print "\nQR = \n", QR
    error = norm(A - QR,inf)
    print "\nInfinity-norm Error between A and QR = ",error
    
    QsQ = dot(Q.T.conj(),Q)   # conjugate transpose for complex tests below
    m,n = A.shape
    error = norm(eye(n) - QsQ, inf)
    print "\nInfinity-norm Error between Q^* Q and I = ",error

In [4]:
A = array([[1,2], [0,1], [1,0]])
test_mgs(A)

Q = 
[[ 0.70710678  0.57735027]
 [ 0.          0.57735027]
 [ 0.70710678 -0.57735027]]

R = 
[[ 1.41421356  1.41421356]
 [ 0.          1.73205081]]

QR = 
[[  1.00000000e+00   2.00000000e+00]
 [  0.00000000e+00   1.00000000e+00]
 [  1.00000000e+00   1.11022302e-16]]

Infinity-norm Error between A and QR =  1.11022302463e-16

Infinity-norm Error between Q^* Q and I =  4.99600361081e-16


In [5]:
A = array([[1.,0], [0,1], [1,0]])
test_mgs(A)

Q = 
[[ 0.70710678  0.        ]
 [ 0.          1.        ]
 [ 0.70710678  0.        ]]

R = 
[[ 1.41421356  0.        ]
 [ 0.          1.        ]]

QR = 
[[ 1.  0.]
 [ 0.  1.]
 [ 1.  0.]]

Infinity-norm Error between A and QR =  0.0

Infinity-norm Error between Q^* Q and I =  2.22044604925e-16


### Test with a larger random matrix:

In [6]:
A = rand(10,6)
test_mgs(A)

Q = 
[[ 0.33431365  0.18403772 -0.33870909  0.67561486 -0.1072167   0.01436242]
 [ 0.3751999  -0.52665614  0.29737101  0.01368883  0.38073111 -0.14234847]
 [ 0.37620499  0.12743861 -0.52126682 -0.14926704  0.52336041 -0.22486812]
 [ 0.00945238  0.00692495  0.10409737  0.41718485  0.18674955  0.32239711]
 [ 0.37987245  0.04308839  0.00506482  0.04834339 -0.63387692 -0.05874415]
 [ 0.20478893  0.20060575  0.66970669  0.29840089  0.12285814 -0.29371557]
 [ 0.38920769  0.12847071  0.10736618 -0.43420033 -0.24072559 -0.23202962]
 [ 0.39322509 -0.07086423 -0.08329467 -0.04021825 -0.04487104  0.06589481]
 [ 0.17589091  0.67828548  0.21405883 -0.22814708  0.23104805  0.4556272 ]
 [ 0.28731879 -0.38571744  0.03836511 -0.11599413 -0.06728585  0.68522211]]

R = 
[[ 2.10493396  1.22879951  1.4405689   1.17516211  1.6331656   1.18715152]
 [ 0.          0.79381106 -0.01938142  0.1487819   0.15495561  0.53469357]
 [ 0.          0.          0.97477797  0.38953494  0.23347866  0.08093454]
 [ 0.        

### Test on many large matrices:

To be even more confident our code is working, we could run many tests...

In [7]:
ntests = 50
m = 100
n = 75
print "Will run %i tests on random %i by %i matrices..." % (ntests,m,n)
maxerror = 0.
for itest in range(ntests):
    A = rand(m,n)
    Q,R = mgs(A)
    QR = dot(Q,R)
    error = norm(A - QR,inf)
    maxerror = max(maxerror, error)
    
print "\nLargest infinity-norm Error between A and QR = ",maxerror

Will run 50 tests on random 100 by 75 matrices...

Largest infinity-norm Error between A and QR =  1.39610545347e-14


## Complex version:

To write a routine that works also for complex A, we need to:

- initialize Q and R to be of complex type
- compute the inner product $q_i^*v_j$, which can be done by:  `inner(Q[:,i].conj(),V[:,j])`

The version below sticks to real arithmetic if the input `A` is real
and uses complex arithmetic otherwise.  The variable `V_dtype` records the type.


In [8]:
def mgs(A):
    """
    Modified Gram-Schmidt for real or complex matrices.
    """
    import copy
    
    # initialize V to be a copy of A, and insure data type is float
    # or complex.  Otherwise convert to float.
    # (in case user initializes with integers):
    if A.dtype not in [float, complex]:
        V = array(A, dtype=float)  
    else:
        V = copy.copy(A)
    V_dtype = V.dtype  # should be float or complex
    print "Arrays are of type %s" % V_dtype
    
    # Initialize Q, R of correct shapes, values filled in later:
    m,n = shape(A)
    Q = zeros((m,n), dtype=V_dtype)
    R = zeros((n,n), dtype=V_dtype)
    
    for i in range(n):
        R[i,i] = norm(V[:,i], 2)
        Q[:,i] = V[:,i]/R[i,i]
        for j in range(i+1,n):
            R[i,j]= inner(Q[:,i].conj(),V[:,j])
            V[:,j] = V[:,j] - R[i,j]*Q[:,i]
            
    return Q,R

### Test on a real matrix:

In [9]:
A = array([[1,2], [0,1], [1,0]])
test_mgs(A)

Arrays are of type float64
Q = 
[[ 0.70710678  0.57735027]
 [ 0.          0.57735027]
 [ 0.70710678 -0.57735027]]

R = 
[[ 1.41421356  1.41421356]
 [ 0.          1.73205081]]

QR = 
[[  1.00000000e+00   2.00000000e+00]
 [  0.00000000e+00   1.00000000e+00]
 [  1.00000000e+00   1.11022302e-16]]

Infinity-norm Error between A and QR =  1.11022302463e-16

Infinity-norm Error between Q^* Q and I =  4.99600361081e-16


### Test on a complex matrix

Note that `1j` is the complex unit in Python ($\sqrt{-1}$).

In [10]:
A = array([[1,2+1j], [3j,1], [1+pi*1j,0]])
test_mgs(A)

Arrays are of type complex128
Q = 
[[ 0.21889855+0.j          0.80346256+0.46238643j]
 [ 0.00000000+0.65669566j  0.30063936-0.1213103j ]
 [ 0.21889855+0.68769009j -0.16747262-0.08659908j]]

R = 
[[ 4.56832621+0.j          0.43779711-0.43779711j]
 [ 0.00000000+0.j          2.36995092+0.j        ]]

QR = 
[[  1.00000000e+00+0.j           2.00000000e+00+1.j        ]
 [  0.00000000e+00+3.j           1.00000000e+00+0.j        ]
 [  1.00000000e+00+3.14159265j   5.55111512e-17+0.j        ]]

Infinity-norm Error between A and QR =  4.99600361081e-16

Infinity-norm Error between Q^* Q and I =  1.98793139177e-16


### Test on  a random complex matrix:

In [11]:
A = rand(10,3) + 1j*rand(10,3)
test_mgs(A)

Arrays are of type complex128
Q = 
[[ 0.34219698+0.08091564j -0.07355067+0.03889236j -0.44023907+0.3958133j ]
 [ 0.29247213+0.20024071j  0.09534998+0.24913126j  0.10795352-0.0073814j ]
 [ 0.26102906+0.35318665j -0.29604848-0.21291661j  0.32513261+0.11356132j]
 [ 0.26354217+0.08263144j  0.11556294+0.15698269j  0.25495252+0.06592937j]
 [ 0.08327772+0.31266275j  0.40165843-0.04164909j  0.08042999-0.14923727j]
 [ 0.29321278+0.27497678j -0.24679147-0.3120602j  -0.13770418+0.07434358j]
 [ 0.01096871+0.13923917j  0.35844788+0.19531055j -0.14674192+0.14760693j]
 [ 0.13210531+0.21738779j  0.14034541+0.13557222j  0.34741535+0.03005618j]
 [ 0.11394889+0.0787244j   0.22675623+0.36432408j -0.19463917+0.19234902j]
 [ 0.23734598+0.23568397j  0.04683041+0.19631642j -0.09709594-0.38392182j]]

R = 
[[ 2.63825654+0.j          2.21332911-0.08117021j  1.94266636+0.229848j  ]
 [ 0.00000000+0.j          1.81323642+0.j          0.69127777+0.15984421j]
 [ 0.00000000+0.j          0.00000000+0.j          1.31481